In [ ]:
%pip install openmander
import openmander as om
from IPython.display import SVG, display

base_path = '/Users/marccanby/Documents/GitRepos/redistricting/packs/'
svg_path = base_path + 'plan.svg'

def display_plan(plan: om.Plan):
    plan.to_svg(svg_path)
    display(SVG(filename=svg_path))

# pack_path = om.download_pack('CT', path=base_path, verbose=1)
pack_path = '/Users/marccanby/Documents/GitRepos/redistricting/packs/CT_2020_pack'
map = om.Map(pack_path)

In [ ]:
import os
import sys
from contextlib import contextmanager

@contextmanager
def silence_fds():
    # Flush any pending output
    sys.stdout.flush()
    sys.stderr.flush()

    # Save original file descriptors
    old_stdout_fd = os.dup(1)
    old_stderr_fd = os.dup(2)

    try:
        # Open devnull and redirect both stdout and stderr
        with open(os.devnull, 'w') as devnull:
            os.dup2(devnull.fileno(), 1)
            os.dup2(devnull.fileno(), 2)

            yield
    finally:
        # Restore original fds
        os.dup2(old_stdout_fd, 1)
        os.dup2(old_stderr_fd, 2)
        os.close(old_stdout_fd)
        os.close(old_stderr_fd)

In [ ]:
plan = om.Plan(map, num_districts=5)


# This returns a list of vertex weight column names:
# for series in plan.series():
#     print(series)

In [ ]:
pop_dev = om.Metric.population_deviation("T_20_CENS_Total")
compactness = om.Metric.compactness_polsby_popper()
competitiveness = om.Metric.competitiveness(dem_series="E_20_PRES_Dem", rep_series="E_20_PRES_Rep", threshold=0.05)
objective = om.Objective(
    metrics=[pop_dev, compactness, competitiveness],
    weights=[0.5, 0.25, 0.25]
)
def score_plan(plan):
    print('\tPop dev:', plan.compute_metric(pop_dev))
    print('\tCompactness:', plan.compute_metric(compactness))
    print('\tCompetitivness:', plan.compute_metric(competitiveness))
    print('\tObjective', plan.compute_objective(objective))

In [ ]:
plan.randomize()

    
# plan.equalize(series="T_20_CENS_Total", tolerance=0.00005, max_iter=1000)
score_plan(plan)
display_plan(plan)

# plan.equalize
# plan.anneal_balance
# plan.tabu_balance
# plan.genetic_optimize


# metric = om.Metric.population_deviation(series="T_20_CENS_Total")
# print(plan.compute_metric(metric))


# print(plan.compute_metric(om.Metric.population_deviation("T_20_CENS_Total")))
# print(plan.compute_metric(om.Metric.compactness_polsby_popper()))


In [ ]:
#with silence_fds():
plan.anneal(objective=objective, max_iter=1_500_000, finish_temp_iter = 1_000_000, initial_temp=100, final_temp=0.01)
score_plan(plan)
display_plan(plan)

In [ ]:
# with silence_fds():
#     plan.anneal_balance(series="T_20_CENS_Total", max_iter=1_000_000, initial_temp=100, final_temp=0.01, boundary_factor=0.01)
# score_plan(plan)
# display_plan(plan)

In [ ]:
# print(plan.compute_metric(om.Metric.population_deviation("T_20_CENS_Total")))
# print(plan.compute_metric(om.Metric.compactness_polsby_popper()))
# competitiveness = om.Metric.competitiveness(dem_series="E_20_PRES_Dem", rep_series="E_20_PRES_Rep", threshold=0)

display_plan(plan)

In [ ]:
plan.anneal_balance(series="T_20_CENS_Total", max_iter=10_000_000, initial_temp=100, final_temp=0.01, boundary_factor=0.01)
print(plan.compute_metric(om.Metric.population_deviation("T_20_CENS_Total")))
print(plan.compute_metric(om.Metric.compactness_polsby_popper()))
display_plan(plan)

In [ ]:
plan.tabu_balance(
    series="T_20_CENS_Total",
    max_iter=10_000,
    tabu_tenure=25,
    boundary_factor=0.25,     # 25% cut length, 75% population balance
    candidates_per_iter=100,  # neighborhood sampling
)

display_plan(plan)

# plan.district_polsby_popper()                   # compactness
# plan.district_vote_share(dem_series="DEM20", rep_series="REP20")
# plan.district_competitiveness(dem_series="DEM20", rep_series="REP20", threshold=0.10)
